In [36]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 500 # reuqests (and passengers)
params.nV = 10 # vehicles

In [38]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.5
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



In [39]:
inData = ExMAS.main(inData, params.shareability, plot=False) # create shareability graph (ExMAS) 

03-07-23 12:53:46-INFO-Initializing pairwise trip shareability between 500 and 500 trips.
03-07-23 12:53:46-INFO-creating combinations
03-07-23 12:53:46-INFO-249500	 nR*(nR-1)
03-07-23 12:53:50-INFO-Reduction of feasible pairs by 99.37%
03-07-23 12:53:50-INFO-Degree 2 	Completed
03-07-23 12:53:50-INFO-trips to extend at degree 2 : 3982
03-07-23 12:53:54-INFO-At degree 2 feasible extensions found out of 1428 searched
03-07-23 12:53:54-INFO-Degree 3 	Completed
03-07-23 12:53:54-INFO-trips to extend at degree 3 : 1428
03-07-23 12:53:56-INFO-At degree 3 feasible extensions found out of 454 searched
03-07-23 12:53:56-INFO-Degree 4 	Completed
03-07-23 12:53:56-INFO-Max degree reached 4
03-07-23 12:53:56-INFO-Trips still possible to extend at degree 4 : 454


In [40]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

03-07-23 12:53:57-INFO-Matching 500 trips to 6364 rides in order to min u_veh
03-07-23 12:53:59-INFO-Problem solution: Optimal. 
Total costs for single trips:        130,843 
reduced by matching to:               91,743


In [41]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],1.467500,79,1,[1.4675],"[0, 79]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1569646491, 1571090443]","[None, 0, 0]",node time req_id od 0 ...,79,0.79,1.191000,0.238200,0.9528
1,[1],5.764000,311,1,[5.764],"[2, 311]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1510592123, 44710267]","[None, 1, 1]",node time req_id od 0 ...,311,3.11,4.675500,0.935100,3.7404
2,[2],6.199000,335,1,[6.199000000000001],"[20, 335]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1608998332, 5124546092]","[None, 2, 2]",node time req_id od 0 ...,335,3.35,5.026500,1.005300,4.0212
3,[3],3.027500,163,1,[3.0275],"[25, 163]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1568976292, 44766586]","[None, 3, 3]",node time req_id od 0 ...,163,1.63,2.457000,0.491400,1.9656
4,[4],6.496500,351,1,[6.496499999999999],"[29, 351]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44723638, 559635312]","[None, 4, 4]",node time req_id od 0 N...,351,3.51,5.268000,1.053600,4.2144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6359,"[132, 145, 138, 152]",19.543875,507,40,"[5.614800000000001, 4.095725, 5.21700000000000...","[982.5, 1, 86, 71, 112, 109, 93, 35]","[132, 145, 138, 152]","[145, 132, 138, 152]",4,6359,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 629532228, 629532203, 44859971, 4486149...","[None, 132, 145, 138, 152, 145, 132, 138, 152]",node time req_id od 0 ...,507,5.07,12.445875,2.489175,9.9567
6360,"[132, 145, 138, 143]",17.717650,497,41,"[5.72855, 4.209474999999999, 5.33075, 2.448875]","[982.5, 1, 86, 45, 137, 26, 109, 93]","[132, 145, 138, 143]","[143, 145, 132, 138]",4,6360,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 629532228, 629532203, 44859971, 3669138...","[None, 132, 145, 138, 143, 143, 145, 132, 138]",node time req_id od 0 ...,497,4.97,11.061000,2.212200,8.8488
6361,"[132, 145, 143, 160]",18.181550,614,40,"[5.68305, 4.027475, 2.280525, 6.1905]","[982.5, 1, 93, 113, 37, 26, 109, 235]","[132, 145, 143, 160]","[143, 145, 132, 160]",4,6361,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 629532228, 629532203, 3669138213, 44873...","[None, 132, 145, 143, 160, 143, 145, 132, 160]",node time req_id od 0 ...,614,6.14,11.693250,2.338650,9.3546
6362,"[132, 145, 143, 152]",16.366250,452,40,"[5.5511, 4.032025, 2.285075, 4.49805]","[982.5, 1, 93, 28, 108, 26, 109, 87]","[132, 145, 143, 152]","[143, 145, 132, 152]",4,6362,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 629532228, 629532203, 3669138213, 44861...","[None, 132, 145, 143, 152, 143, 145, 132, 152]",node time req_id od 0 ...,452,4.52,10.656000,2.131200,8.5248


In [ ]:
responses = []
avg_kpi = []
idle_time = []

for i in range(3, 4):
    params.kpi = i
    sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/pax{}.csv'.format(i))
    #driver_data.loc['Cost'] = driver_data.loc['Revenue'].apply(lambda x: x*params.shareability.operating_cost)['Vehicles'] = sim.res[0].veh_exp.index
    #sim.res[0].veh_exp['ds'] = f"{i}"
    
    #responses.append(sim.res[0].veh_exp)
     
   # vehicles = sim.res[0].veh_exp.loc[sim.res[0].veh_exp["nRIDES"] > 0]
    #no_of_veh = len(vehicles)
    
    #avg_kpi.append(sim.res[0].all_kpi/no_of_veh)
    #idle_time.append(vehicles['IDLE'].sum()/no_of_veh)
    
    #_inData.sblts.rides['commission'] = _inData.sblts.rides.apply(lambda row: row.fare*sp.comm_rate, axis=1)
    

03-07-23 12:55:12-INFO-Matching 500 trips to 6364 rides in order to min u_veh
03-07-23 12:55:15-INFO-Problem solution: Optimal. 
Total costs for single trips:        130,843 
reduced by matching to:               91,743
0.5
0.5
03-07-23 12:56:25-CRITICAL-this is request 1 with [1, 1103, 2906] available rides.
03-07-23 12:56:25-CRITICAL-ride 1 available [1]
03-07-23 12:56:25-CRITICAL-ride 1103 available [1, 0]
03-07-23 12:56:25-CRITICAL-ride 2906 available [1, 0]
03-07-23 12:56:25-CRITICAL-this is reuqest 1 with [1, 1103, 2906] still available rides.
private rides
03-07-23 12:56:25-CRITICAL-vehicle 6 has 3 choices
0.5
03-07-23 12:56:25-CRITICAL-this is request 2 with [2, 741, 893, 959, 1156, 1184, 2641, 2789, 4429, 4430, 4431, 4432, 4433, 4434, 4435, 4436, 4437, 5224, 5225, 5226, 5891, 5892, 6139, 6140] available rides.
03-07-23 12:56:25-CRITICAL-ride 2 available [1]
03-07-23 12:56:25-CRITICAL-ride 741 available [1, 0]
03-07-23 12:56:25-CRITICAL-ride 893 available [1, 0]
03-07-23 12:56: